[View in Colaboratory](https://colab.research.google.com/github/thanhptse/DataMining/blob/master/HomeWork1.ipynb)

# **1. Data Understanding**

### **1.1 Import libary**

In [0]:
import pandas as pd
import io
from google.colab import files

### 1.2 Setup helper Functions

In [0]:
def get_street(row):
  str_join = " "
  street_name = row['address'].split(',')[0].split(' ')
  if len(street_name) > 1 and street_name[0].isdigit():
    street_name.pop(0)
    
  return str_join.join(street_name)

### **1.3 Load data**

In [137]:
# update 2 file Employee_Addresses.csv and Potentail_Bust_Stops.csv
uploaded = files.upload()

Saving Employee_Addresses.csv to Employee_Addresses (1).csv
Saving Potentail_Bust_Stops.csv to Potentail_Bust_Stops (1).csv


In [0]:
df_employee_address = pd.read_csv(io.StringIO(uploaded['Employee_Addresses.csv'].decode('utf-8')))
df_potentail_bust_stop = pd.read_csv(io.StringIO(uploaded['Potentail_Bust_Stops.csv'].decode('utf-8')))


### 1.4 Statistical summaries and visualisations

Start look few lines of data

In [139]:
df_employee_address.head()

,address,employee_id
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206
1,"237 Accacia St, Daly City, CA 94014, USA",2081
2,"1835 Folsom St, San Francisco, CA 94103, USA",178
3,"170 Cambridge St, San Francisco, CA 94134, USA",50
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863


In [140]:
df_employee_address['employee_id'].count()

2191

We know we have address of 2191 employees

In [141]:
df_potentail_bust_stop.head()

,Street_One,Street_Two
0,MISSION ST,ITALY AVE
1,MISSION ST,NEW MONTGOMERY ST
2,MISSION ST,01ST ST
3,MISSION ST,20TH ST
4,MISSION ST,FREMONT ST


In [142]:
df_potentail_bust_stop.groupby('Street_One').count()

,Street_Two
Street_One,
MISSION ST,119


One thing special is MISSION ST intersects with all other street

# **2. Data Preparation**

In [145]:
df_employee_address['only_street_name'] = df_employee_address.apply (lambda row: get_street(row),axis=1)
df_employee_address.head()

,address,employee_id,only_street_name
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206,Edinburgh St
1,"237 Accacia St, Daly City, CA 94014, USA",2081,Accacia St
2,"1835 Folsom St, San Francisco, CA 94103, USA",178,Folsom St
3,"170 Cambridge St, San Francisco, CA 94134, USA",50,Cambridge St
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863,Roanoke St


We split address to get only street name

In [146]:
df_popular_street = df_employee_address.groupby('only_street_name')['only_street_name'].count().reset_index(name='count').sort_values(['count'], ascending=False)
df_popular_street.head(15)

,only_street_name,count
11,16th St,61
115,Geneva Ave,50
35,Alemany Blvd,47
111,Folsom St,46
261,Twin Peaks Blvd,42
166,Mission St,42
237,Silver Ave,36
230,Schwerin St,36
157,Market St,32
9,15th St,30


Get list most road popular of employees

In [147]:
df_potentail_bust_stop['street_name_lower'] = df_potentail_bust_stop['Street_Two'].str.lower().str.strip()
df_potentail_bust_stop.head()

,Street_One,Street_Two,street_name_lower
0,MISSION ST,ITALY AVE,italy ave
1,MISSION ST,NEW MONTGOMERY ST,new montgomery st
2,MISSION ST,01ST ST,01st st
3,MISSION ST,20TH ST,20th st
4,MISSION ST,FREMONT ST,fremont st


Lower street name and remove white space of bust stop

In [148]:
df_popular_street['street_name_lower'] = df_popular_street['only_street_name'].str.lower().str.strip()
df_popular_street.head()

,only_street_name,count,street_name_lower
11,16th St,61,16th st
115,Geneva Ave,50,geneva ave
35,Alemany Blvd,47,alemany blvd
111,Folsom St,46,folsom st
261,Twin Peaks Blvd,42,twin peaks blvd


Lower street name and remove white space list most road popular of employee

# **3. Evaluation **

In [149]:
result = pd.merge(df_potentail_bust_stop, df_popular_street, on='street_name_lower', how='inner').sort_values(['count'], ascending=False).head(10)
result[['Street_One', 'Street_Two', 'count']]

,Street_One,Street_Two,count
32,MISSION ST,16TH ST,61
10,MISSION ST,GENEVA AVE,50
4,MISSION ST,SILVER AVE,36
6,MISSION ST,15TH ST,30
21,MISSION ST,14TH ST,23
31,MISSION ST,PERSIA AVE,18
33,MISSION ST,BOSWORTH ST,16
22,MISSION ST,10TH ST,12
1,MISSION ST,13TH ST,11
29,MISSION ST,ROLPH ST,11


After join 2 dataframe with street name we got the best 10 stops.

The algorithm finds all the streets with the most employees on it, and then checks whether those streets are on the bus stops.
Algorithms may be inaccurate when there are identical name. Because it is based only on the name of the street, it is sometimes incorrect to calculate the distance between employees' addresses and compare it to a bus stop.